In [6]:
# #@title Download Kaggle API token from Google Drive
# import gdown
# import os
# from pathlib import Path

# gdrive_url = 'https://drive.google.com/uc?id=1P6t7O8vOjmQi3rMX0KQlgO6BT0fk2dI-'
# if not os.path.exists('kaggle.json'):
#   gdown.download(gdrive_url, 'kaggle.json', quiet=False)

# kaggle_dir = Path.home() / '.kaggle'
# kaggle_token_path = kaggle_dir / 'kaggle.json'

# if not kaggle_token_path.exists():
#   kaggle_dir.mkdir(parents=True, exist_ok=True)
#   os.rename('kaggle.json', kaggle_token_path)
#   kaggle_token_path.chmod(0o600)

# dataset_path = 'global_covid-19'
# if not os.path.exists(dataset_path):
#   os.mkdir(dataset_path)
#   !kaggle datasets download -d rohitashchandra/global-covid19-twitter-dataset --path {dataset_path} --unzip
#   gdrive_url = 'https://drive.google.com/uc?id=19Hr0PQSyqGqLLJlPaDKjMfvlbFVnYbBr'
#   os.remove(os.path.join(dataset_path, 'India.csv'))
#   gdown.download(gdrive_url, os.path.join(dataset_path, 'India.csv'), quiet=False)

In [7]:
#@title display csv as dataframes

import os
import pandas as pd
import random
from csv import QUOTE_NONE

dataset = ['Australia.csv', 'Brazil.csv', 'India.csv', 'Indonesia.csv', 'Japan.csv']

for data in dataset:
  try:
    df = pd.read_csv(data, encoding='utf-8', on_bad_lines='skip', quoting=QUOTE_NONE)
  except UnicodeDecodeError:
    df = pd.read_csv(data, encoding='ISO-8859-1', on_bad_lines='skip', quoting=QUOTE_NONE)

  # Fetch 10 random rows
  random_rows = df.sample(n=3)

  # Display the random rows
  print(f"Random rows from {data}:")
  display(random_rows)

Random rows from Australia.csv:


,Unnamed: 0,created_at,text,user_location
265140,2,"000 of those tests have returned positive.…""","""Noosa","Queensland"""
255372,337467,Mon Dec 06 04:53:43 +0000 2021,"""RT @GiselleLucasx: Let's celebrate the end of...",NaN
257136,Come and see us tonight and get immediate #men...,NaN,NaN,NaN


Random rows from Brazil.csv:


,Unnamed: 0,created_at,text,user_location
119237,Mon Dec 20 13:40:43 +0000 2021,RT @ArleneDickinson: Hey governments!! Drop so...,"""Nova Scotia","Canada"""
"#NoVaccineNoVacancy""",Ghazipur Uttar Pradesh,NaN,NaN,NaN
– Total PCR Tests : 5,740,182,NaN,NaN


Random rows from India.csv:


,,Unnamed: 0,created_at,text,user_location
673823,Mon Oct 19 12:57:23 +0000 2020,Study shows how people can reduce stress durin...,India,NaN,NaN
Primary Number : 9967692340,NaN,NaN,NaN,NaN,NaN
#follow us to stay updated,NaN,NaN,NaN,NaN,NaN


Random rows from Indonesia.csv:


,Unnamed: 0,created_at,text,user_location
140816,#IndonesiaMatters,NaN,NaN,NaN
161236,140387,Sun Sep 05 01:42:43 +0000 2021,"""RT @ashishkjha: Close family member just call...",NaN
194403,Read more… https://t.co/Ztt1RUQ8dR #Philippine...,Jakarta,NaN,NaN


Random rows from Japan.csv:


,,,Unnamed: 0,created_at,text,user_location
185255,Fri Aug 13 12:07:45 +0000 2021,"""RT @awgaffney: If we're going to roll out booster shots to entire population",shouldn't we prove that they work in preventi...,Manitoba,NaN,NaN
"If y…""","""Silverstone","Northamptonshire""",NaN,NaN,NaN,NaN
391696,Mon Aug 30 14:22:14 +0000 2021,"""RT @RepThomasMassie: """"I wanted to serve my country and now I am looking at a result in which I am lumped in with felons and rapists","I won'…""",Truth Social @OldManStoneZone,NaN,NaN


In [8]:
#@title display csv as dataframes

import os
import pandas as pd
import random
from csv import QUOTE_NONE
import csv


for data in dataset:
  try:
    with open(data, 'r', encoding='utf-8', newline='') as file:
            df = pd.read_csv(file, on_bad_lines='skip', quoting=csv.QUOTE_MINIMAL)
  except UnicodeDecodeError:
    with open(data, 'r', encoding='ISO-8859-1', newline='') as file:
            df = pd.read_csv(file, on_bad_lines='skip', quoting=csv.QUOTE_MINIMAL)

  # Fetch 10 random rows
  random_rows = df.sample(n=3)

  # Display the random rows
  print(f"Random rows from {data}:")
  display(random_rows)

Random rows from Australia.csv:


,Unnamed: 0,created_at,text,user_location
41595,41595,Fri Dec 10 09:07:00 +0000 2021,RT @LozzaFox: I would like to see this man’s f...,warwickshire
151769,151769,Wed Dec 29 06:36:52 +0000 2021,RT @SandroDemaio: Masks are a small price for ...,"New South Wales, Australia"
30927,30927,Wed Dec 08 23:50:11 +0000 2021,RT @MichelleRoseG33: It’s my #friday I have to...,Kingston upon Hull


Random rows from Brazil.csv:


,Unnamed: 0,created_at,text,user_location
134746,134746,Wed Dec 22 16:48:52 +0000 2021,@davidhogg111 Harvard must be a joke now. How...,Sacramento
177673,177673,Thu Sep 02 22:05:32 +0000 2021,RT @larryelder: “The documents—mostly emails—s...,Brasil
366109,366109,Wed Mar 17 19:09:17 +0000 2021,I am blind. I have Charles Bonnet syndrome. It...,"Halifax, Nova Scotia"


Random rows from India.csv:


,Unnamed: 0,created_at,text,user_location
9805,345928,Mon Oct 19 09:32:01 +0000 2020,A New Zealand journalist is being praised arou...,"West Bengal, India"
3644,237310,Mon Oct 19 08:00:12 +0000 2020,#gstamnesty\n#GSTAmnesty \n\n@nsitharamanoffc ...,"Karjat, India"
31069,975175,Mon Oct 19 15:23:37 +0000 2020,PLS THIS IS LITERALLY THE WORLD IN THE PANDEMI...,India


Random rows from Indonesia.csv:


,Unnamed: 0,created_at,text,user_location
106739,106739,Mon Feb 01 23:13:44 +0000 2021,RT @TokyoFashion: Art director Sebastian Masud...,Jakarta Capital Region
64482,64482,Thu Aug 05 10:04:01 +0000 2021,RT @svpino: We were not ready.\n\nWhen the pan...,Indonesia
190951,190951,1351094531194839040,RT @bambiplushie: i see some of you havent rea...,Indonesia


Random rows from Japan.csv:


,Unnamed: 0,created_at,text,user_location
31871,31871,Wed Dec 29 19:55:43 +0000 2021,Thank god. https://t.co/1GDcTlYhRL,"Winnipeg, Manitoba"
234066,234066,Wed Feb 24 20:20:03 +0000 2021,RT @SenSchumer: We are working to quickly pass...,Santa Monica/Fukuoka
259969,259969,Fri Mar 12 06:10:18 +0000 2021,@Christina_Mitas @archtoronto @fordnation Ever...,Manitoba


In [10]:
#@title merge into one csv

import pandas as pd
from csv import QUOTE_NONE

# List to hold the dataframes
dataframes = []

# Read each CSV file with specific settings to handle quoting issues
for data in dataset:
  try:
    # Open the file with utf-8 encoding, handle errors by replacing problematic characters
    with open(data, 'r', encoding='utf-8', newline='') as file:
      df = pd.read_csv(file, on_bad_lines='skip', quoting=csv.QUOTE_MINIMAL)
      df = df.iloc[:, 1:]
  except UnicodeDecodeError:
    # Fallback to a different encoding if UTF-8 doesn't work
    with open(data, 'r', encoding='ISO-8859-1', newline='') as file:
      df = pd.read_csv(file, on_bad_lines='skip', quoting=csv.QUOTE_MINIMAL)
      df = df.iloc[:, 1:]
  dataframes.append(df)

# Concatenate all dataframes
combined_df = pd.concat(dataframes, ignore_index=True)

# Save the combined dataframe to a new CSV file
combined_df.to_csv('tweets.csv', index=False)


# open and sample 20 rows from tweets.csv
df = pd.read_csv('tweets.csv')
random_rows = df.sample(n=20)
print(f"Random rows from {df}:")
display(random_rows)

Random rows from                              created_at  \
0        Wed Dec 08 04:25:46 +0000 2021   
1        Wed Dec 08 04:25:57 +0000 2021   
2        Wed Dec 08 04:26:06 +0000 2021   
3        Wed Dec 08 04:26:07 +0000 2021   
4        Wed Dec 08 04:26:07 +0000 2021   
...                                 ...   
1510291  Wed Sep 08 17:46:54 +0000 2021   
1510292  Wed Sep 08 17:46:59 +0000 2021   
1510293  Wed Sep 08 17:47:22 +0000 2021   
1510294  Wed Sep 08 17:47:32 +0000 2021   
1510295  Wed Sep 08 17:47:36 +0000 2021   

                                                      text  \
0        RT @AaronSiriSG: Litigation update:  \n\nFDA d...   
1        RT @covidbaseau: The TGA has approved the Mode...   
2        RT @SimoLove: Federal Health Minister Greg Hun...   
3        @twityouse Are these the deniers that are wear...   
4        RT @drsimonegold: WOW: a preprint study for Th...   
...                                                    ...   
1510291  @Taleeb Your boss break

,created_at,text,user_location
710922,Fri Feb 26 10:40:22 +0000 2021,RT @BTS_UPDATES_GER: [ENG TRANSLATION]\n\ntw /...,"Japeri, Brasil"
647398,Mon Aug 09 12:45:59 +0000 2021,RT @EY_Cyprus: According to the #EYCyprus Busi...,"Barranquilla, Colombia"
813584,Mon Feb 01 18:43:26 +0000 2021,RT @briebriejoy: I reached out to NNU &amp; wa...,"São Carlos, Brasil"
707681,Thu Feb 25 01:13:48 +0000 2021,In the same way that white Republican males pr...,Santa Cruz
1086140,Fri May 08 11:43:01 +0000 2020,RT @tzimmer_history: Here’s a story about how ...,"nipaluna, palawa land"
194399,Fri Dec 31 06:13:49 +0000 2021,RT @JohnRuddick2: 2020: Year of COVID-mania.\...,"Victoria, Australia"
1113898,Sat Dec 25 02:30:29 +0000 2021,RT @Trinifood: What did the unions say?,Trinidad and Tobago
1446498,Sat Mar 27 03:18:07 +0000 2021,RT @TheBreakdownAB: From the DM’s...\n\nAs a f...,Manitoba
1043695,Mon Dec 20 09:18:21 +0000 2021,RT @rishipuff: when the pandemic started i was...,"Bali, Indonesia"
1383961,Wed Jun 23 06:16:04 +0000 2021,@SimonKaggwaNjal @MinofHealthUG @SimonKaggwaNj...,Girl from Kazo


In [12]:
lexicon = pd.read_csv('NRC-Emotion-Lexicon-Wordlevel-v0.92.txt', sep='\t', names=['word', 'emotion', 'association'])
lexicon_pivot = lexicon.pivot(index='word', columns='emotion', values='association').reset_index()

In [13]:
def detect_emotion(text):
    words = text.split()
    emotions_score = {'worried': 0, 'happy': 0, 'neutral': 0, 'angry': 0}

    for word in words:
        if word in lexicon_pivot['word'].values:
            word_data = lexicon_pivot[lexicon_pivot['word'] == word]
            for emotion in ['worried', 'happy', 'neutral', 'angry']:
                # Here you can define how each lexicon's emotion category maps to your categories
                if emotion in word_data.columns and word_data.iloc[0][emotion] == 1:
                    emotions_score[emotion] += 1

    return max(emotions_score, key=emotions_score.get)

In [14]:
df['text'] = df['text'].astype(str)
df['emotion'] = df['text'].apply(detect_emotion)
df.to_csv('labeled_tweets.csv', index=False)


In [15]:
df['emotion'].value_counts()

emotion
worried    1510296
Name: count, dtype: int64

In [17]:
lexicon_pivot.head(100)

emotion,word,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust
0,NaN,0,0,0,0,0,0,0,0,0,0
1,aback,0,0,0,0,0,0,0,0,0,0
2,abacus,0,0,0,0,0,0,0,0,0,1
3,abandon,0,0,0,1,0,1,0,1,0,0
4,abandoned,1,0,0,1,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
95,accomplishment,0,0,0,0,0,0,1,0,0,0
96,accord,0,0,0,0,0,0,1,0,0,1
97,accordance,0,0,0,0,0,0,0,0,0,0
98,accordion,0,0,0,0,0,0,0,0,0,0


In [18]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

# Load and prepare the lexicon
lexicon = pd.read_csv('NRC-Emotion-Lexicon-Wordlevel-v0.92.txt', sep='\t', names=['word', 'emotion', 'association'])
lexicon_pivot = lexicon.pivot(index='word', columns='emotion', values='association').fillna(0)

# Define a more nuanced emotion mapping
emotion_mapping = {
    'anger': ['anger'],
    'fear': ['fear'],
    'joy': ['joy'],
    'worried': ['fear', 'sadness'],
    'sadness': ['sadness'],
    'disgust': ['disgust'],
    'trust': ['trust'],
    'anticipation': ['anticipation'],
    'anxiety': ['fear', 'anticipation', 'negative']  # New definition for anxiety
}

# Vectorize tweets
vectorizer = CountVectorizer(vocabulary=lexicon_pivot.index)
tweet_matrix = vectorizer.fit_transform(df['text'])

# Calculate emotion scores
emotion_scores = np.zeros((data.shape[0], len(emotion_mapping)))

for i, (category, emotions) in enumerate(emotion_mapping.items()):
    indices = np.array([lexicon_pivot.columns.get_loc(emotion) for emotion in emotions if emotion in lexicon_pivot.columns])
    emotion_scores[:, i] += tweet_matrix.dot(lexicon_pivot.iloc[:, indices].sum(axis=1))

# Normalize scores
emotion_scores = emotion_scores / emotion_scores.sum(axis=1, keepdims=True)

# Assign the dominant emotion
df['emotion'] = pd.DataFrame(emotion_scores, columns=emotion_mapping.keys()).idxmax(axis=1)

# Save results
df.to_csv('labeled_tweets.csv', index=False)


FileNotFoundError: [Errno 2] No such file or directory: 'path_to_your_data.csv'